# BIDSification

Run heudiconv: details in **code/01heudiconv.ipynb**

# Exclude dummy TRs

Once you get the data in BIDS format, you need to get rid of the dummy TRs at the beginning of the run BEFORE running FMRIPREP

Run **/Shared/rblock_mr/code/02exclude_first_TRs.sh** 

Rest: discarded 5TRs (240 --> 235)
Sternberg and Hippo: discarded 3TRs (333 --> 330)

Add this to the json file:

"NumberOfVolumesDiscardedByUser": XX,

# Add Slice Timing information to all json files

of rest, sternberg and hipp tasks

In [ ]:
%% bash

for sub in 8062 ; do
for task in rest hippo sternberg ; do

sed -i 40r<(sed '1!d' /Shared/rblock_mr/rawdata/slicetiming.txt) /Shared/rblock_mr/rawdata/sub-${sub}/func/sub-${sub}_task-${task}_bold.json

done
done


# Run FMRIPREP

Create the template file below with SUBJECT as a placeholder.

In [ ]:
# Run FMRIPREP

# Pull the lastest version of fmriprep

singularity pull docker://poldracklab/fmriprep:latest
        
# Run FMRIPREP on the cluster -- first create a template

#!/bin/sh
#$ -pe smp 10
#$ -q PINC
#$ -m e
#$ -M tien-tong@uiowa.edu
#$ -o /Users/tientong/logs/block/fmriprep/out
#$ -e /Users/tientong/logs/block/fmriprep/err
OMP_NUM_THREADS=8
singularity run -H /Users/tientong/singularity_home \
/Users/tientong/poldracklab_fmriprep_1.4.1-2019-07-09-86bf8bc4b7d5.img \
/Shared/rblock_mr/rawdata \
/Shared/rblock_mr/derivatives \
participant --participant-label SUBJECT --skip_bids_validation \
--fs-license-file /Shared/oleary/functional/FreeSurferLicense/license.txt \
-w /Shared/rblock_mr/derivatives --write-graph \
--use-aroma --error-on-aroma-warnings --output-spaces T1w MNI152NLin6Asym:res-2 --stop-on-first-crash \
--omp-nthreads 8 --nthreads 8 --mem_mb 22500 --notrack

Use `sed` to replace the placeholder with actual IDs, then submit all jobs to queue

In [ ]:
for sub in $(cat /Shared/rblock_mr/sourcedata/subjectlist.txt | tr '\n' ' ') ; do
sed -e "s|SUBJECT|${sub}|" fmriprep_TEMPLATE.job > fmriprep_sub-${sub}.job
done


for sub in 8006 8013 8016 8018 8019 8021 8022 8023 8024 8025 8028 8029 8030 8031 8034 8036 8041 8042 8043 8045 8046 8047 8048 8049 8050 8052 8054 8056 8058 8059 8060 8061 8062 8070 ; do
sed -e "s|SUBJECT|${sub}|" mriqc_TEMPLATE.job > mriqc_sub-${sub}.job
done

for sub in 8006 8013 8016 8018 8019 8021 8022 8023 8024 8025 8028 8029 8030 8031 8034 8036 8041 8042 8043 8045 8046 8047 8048 8049 8050 8052 8054 8056 8058 8059 8060 8061 8062 8070 ; do
qsub mriqc_sub-${sub}.job
done


for sub in $(cat /Shared/rblock_mr/sourcedata/subjectlist.txt | tr '\n' ' ') ; do
    qsub fmriprep_sub-${sub}.job
done 


for sub in 8016 8019 8021 8022 8028 8030 8031 8034 8036 8041 8042 8043 8045 8046 8047 ; do
    qsub fmriprep_sub-${sub}.job
done

# Edit confounds files

Confounds files are in:

```/Shared/rblock_mr/derivatives/fmriprep/TASK/fmriprep/SUB/func```

We need to create a new confound file to include only the confounds we want (named `sub-SUB_ses-SES_task-TASK_run-RUN_bold_confounds-use.tsv`).

AS OF SEPT 17 2018: the results from FMRIPREP have already been motion corrected, slice timing corrected, smoothed, and denoised non-aggressively with ICA_AROMA. The confound file will be the 6 motion parameters.

The script to edit confounds files is:

`/Shared/rblock_mr/code/edit_confounds.R`


# Edit event files

We first have to extract behavioral data from txt file (E-Prime, presentation, etc.) to BIDS formatted event files (BehavioralData/eventtiming.R), then copy to `func`

In [ ]:
%%bash
for sub in 8006 8013 8016 8018 8019 8021 8022 8023 8024 8025 8028 8029 8030 8031 8034 8036 8041 8042 8043 8045 8046 8047 8048 8049 8050 8052 8054 8056 8058 8059 8060 8061 8062 8070 ; do

#for sub in 8062 ; do

cp /Shared/rblock_mr/sourcedata/timingfiles/sternberg/sub-${sub}_task-sternberg_events.tsv /Shared/rblock_mr/rawdata/sub-${sub}/func/

cp /Shared/rblock_mr/sourcedata/timingfiles/hippo/sub-${sub}_task-hippo_events.tsv /Shared/rblock_mr/rawdata/sub-${sub}/func/

done

Then, convert BIDS event files to 3 columns formatted event files, using 
    
`/Shared/rblock_mr/code/03BIDSto3col-allsub_sternberg.sh`
`/Shared/rblock_mr/code/03BIDSto3col-allsub_hippo.sh`  

# Run first level GLM for each subject for each run

## Create a template GLM via Feat GUI:

__For FMIRPREP output AFTER ICA AROMA__
    
    Code saved in: /Shared/rblock_mr/code/subject_glm/hippo and /Shared/rblock_mr/code/subject_glm/sternberg
    Results saved in: /Shared/rblock_mr/derivatives/subject_glm/hippo and /Shared/rblock_mr/derivatives/subject_glm/hippo

    
### Feat GUI Data tab
4D input data

For FMIRPREP output AFTER ICA AROMA == same as FMIRPREP output after ICA AROMA but no Motion parameters

`/Shared/oleary/functional/UICL/BIDS/derivatives/fmriprep/stopsignal/fmriprep/sub-3003/ses-60844114/func/sub-3003_ses-60844114_task-stopsignal_run-1_bold_space-MNI152NLin2009cAsym_variant-smoothAROMAnonaggr_preproc.nii.gz`



Output directory: 

__For FMIRPREP output AFTER ICA AROMA__

`/Shared/oleary/functional/UICL/BIDS/derivatives/subject_level_glm/6regressors`


Total volumes: 205-5 = 200 (exclude the first 5 TRs)

TR: 2.8s

Delete volumes: 0

High pass filter cutoff: 100s (= .01 Hz)

### Feat GUI Pre-stats tab

Turn off everything except BET and High pass (ICA aroma pipeline already smooth the data

### Feat GUI Registration tab

Change to 6DOF - faster registration, because we don't care about the result of registration anyway. The 4D input was already registered to MNI

### Feat GUI Stats tab

Use FILM prewhitening

#### Confounds 

Have to run the following scripts to get the confound file:

1. Create an unsmooth denoised BOLD `fmri01_aromaunsmooth.sh`	
2. Calculate the average signal from CSF and WM of the unsmooth denoised BOLD `fmri02_WMCSFts.sh`	
3. Put the CSF and WM signal into a confound file-- to be regressed out later in FEAT `fmri03_confound.R`
    
### Feat GUI Post-stats tab

Uncorrected Thresholding + turn off Create time series plots

## Save this set up as a template `subject_glm_TEMPLATE.fsf`


Make sure that you have turned off motion correction, slice time correction.


## Write subject specific Feat then run it

Create subject specific Feat using the bash code below


In [ ]:
%%bash 

sub=(8024 8045 8070)
dir=/Shared/rblock_mr/code/subject_glm/hippo


for n in 0 1 2 ; do
sed -e "s|SUBJECT|${sub[$n]}|g" $dir/sub-TEMPLATE.fsf > $dir/sub-${sub[$n]}.fsf
done

###########

sub=(8024 8045 8070)
dir=/Shared/rblock_mr/code/subject_glm/sternberg


for n in 0 1 2 ; do
sed -e "s|SUBJECT|${sub[$n]}|g" $dir/sub-TEMPLATE.fsf > $dir/sub-${sub[$n]}.fsf
done


Run the .fsf script with:
`feat SCRIPTNAME.fsf` 

Run multiple scripts at once to speed things up.

`ls *sub-8*.fsf | xargs -i{} -P10 feat {}`

# Second level GLM 

## Delete unnecessary registration files

Since outputs from FMRIPREP were already registerd to MNI, we need to stop FSL from doing registration one more time.

To do this, run 

`/Shared/rblock_mr/code/skip_registration.sh`


This script will make modification in the `reg/` directory of Feat outputs. In particular, it will replace the EPI --> MNI matrix by an identity matrix to make sure the there would be no transformation. Secondly, it will replace the standard.nii.gz by a random EPI volume, to make sure that there would be no interpolation.

For more information, watch this: https://www.youtube.com/watch?v=U3tG7JMEf7M 


## Make 2nd level .fsf script

### Data tab

Hippo: Inputs are 3d Cope image: choose cope 1 (AL vs SI) and cope 2 (RE vs SI)
Sternberg: Inputs are 3d Cope image: choose cope 1 (PT vs CT) and cope 2 (NT vs CT)

### Stats tab

Choose Fixed effects model.

Model setup: EV1 = AL vs SI, EV2 = RE vs SI
Contrast = EV1 - EV2 and EV2 - EV1 

### Post-stats tab

Uncorrected thresholding

## Save this set up as a template `avg_across_runs_TEMPLATE.fsf`

script in 

/Shared/rblock_mr/code/subject_glm/hippo/ALvsRE
/Shared/rblock_mr/code/subject_glm/sternberg/PTvsNT


# Create subject specific fsf file from template

Then run multiple scripts in parallel using xargs

In [ ]:
%%bash 

sub=(8006 8013 8016 8018 8019 8021 8022 8023 8024 8025 8028 8029 8030 8031 8034 8036 8041 8042 8043 8045 8046 8047 8048 8049 8050 8052 8054 8056 8058 8059 8060 8061 8070)
dir=/Shared/rblock_mr/code/subject_glm/hippo/ALvsRE


for n in 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 ; do
sed -e "s|SUBJECT|${sub[$n]}|g" $dir/sub-TEMPLATE.fsf > $dir/sub-${sub[$n]}.fsf
done


###########

sub=(8006 8013 8016 8018 8019 8021 8022 8023 8024 8025 8028 8029 8030 8031 8034 8036 8041 8042 8043 8045 8046 8047 8048 8049 8050 8052 8054 8056 8058 8059 8060 8061 8070)
dir=/Shared/rblock_mr/code/subject_glm/sternberg/PTvsNT

for n in 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 ; do
sed -e "s|SUBJECT|${sub[$n]}|g" $dir/sub-TEMPLATE.fsf > $dir/sub-${sub[$n]}.fsf
done

**When done with subject level analysis**, run `fslmaths` to combine positive and negative stat maps

In [ ]:
%%bash

sub=(8006 8013 8016 8018 8019 8021 8022 8023 8024 8025 8028 8029 8030 8031 8034 8036 8041 8042 8043 8045 8046 8047 8048 8049 8050 8052 8054 8056 8058 8059 8060 8061 8070)
sternberg=/Shared/rblock_mr/derivatives/subject_glm/sternberg/PTvsNT/

for n in 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 ; do
cd $sternberg/sub-${sub[$n]}.gfeat/cope1.feat
fslmaths thresh_zstat2.nii.gz -mul -1 thresh_zstat2_neg.nii.gz
fslmaths thresh_zstat1.nii.gz -add thresh_zstat2_neg.nii.gz thresh_zstat_combined.nii.gz

done


###########

sub=(8006 8013 8016 8018 8019 8021 8022 8023 8024 8025 8028 8029 8030 8031 8034 8036 8041 8042 8043 8045 8046 8047 8048 8049 8050 8052 8054 8056 8058 8059 8060 8061 8070)
hippo=/Shared/rblock_mr/derivatives/subject_glm/hippo/ALvsRE

for n in 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 ; do
cd $hippo/sub-${sub[$n]}.gfeat/cope1.feat
fslmaths thresh_zstat2.nii.gz -mul -1 thresh_zstat2_neg.nii.gz
fslmaths thresh_zstat1.nii.gz -add thresh_zstat2_neg.nii.gz thresh_zstat_combined.nii.gz
done

In [ ]:
# Check the result to see if MTL/hippocampus was activated

afni /Shared/pinc/sharedopt/apps/fsl/Linux/x86_64/5.0.8_multicore/data/standard/MNI152_T1_2mm_brain.nii.gz thresh_zstat_combined.nii.gz